# Hex-level GridScore data prep

Goal: build a merged dataframe that attaches region-level GridScore components to each hex cell and keeps the per-hex climate/terrain fields (temp, elevation). We'll reuse existing CSVs only:

- `datacenter_scores_real.csv`: region-level metrics (profitability, sustainability, dc_score, and raw/normalized inputs)
- `hex_weather_data_all.csv`: per-hex climate/terrain (hex_id, local_temp_c, elevation_m)
- Optionally, we can bring in precomputed fields from the existing map JSON if needed later.

**This notebook stops at the merged dataframe**; no JSON/GeoJSON export yet.


In [1]:
import json
from pathlib import Path

import pandas as pd

DATA_DIR = Path('.')
SCORES_PATH = DATA_DIR / 'datacenter_scores_real.csv'
HEX_CLIMATE_PATH = DATA_DIR / 'hex_weather_data_all.csv'

assert SCORES_PATH.exists(), 'datacenter_scores_real.csv not found'
assert HEX_CLIMATE_PATH.exists(), 'hex_weather_data_all.csv not found'


In [2]:
# Quick peek at the inputs
scores_df = pd.read_csv(SCORES_PATH)
hex_df = pd.read_csv(HEX_CLIMATE_PATH)

scores_df.head(), hex_df.head()

(  region   lat    lon  raw_price  raw_load  raw_volatility  raw_peak  \
 0     NY  42.9  -75.3    47.0075   18803.0      651.338754   20325.0   
 1    CAR  35.5  -80.0    55.9950   22398.0      688.416472   35825.0   
 2    TEN  36.0  -86.0    44.8175   17927.0      417.266953   26174.0   
 3     NW  45.5 -120.5   114.7850   45914.0     2068.570205   49728.0   
 4     NE  42.5  -72.5    38.5175   15407.0      640.962908   16897.0   
 
    raw_renew   raw_temp   n_price    n_load  n_volatility    n_temp   n_renew  \
 0  51.350038   5.800000  0.919927  0.919927      0.936484  1.000000  0.804842   
 1  60.187159  13.795522  0.877463  0.877463      0.926423  0.483513  1.000000   
 2  50.679782  13.898507  0.930274  0.930274      1.000000  0.476861  0.790040   
 3  55.439846   8.534328  0.599693  0.599693      0.551913  0.823371  0.895161   
 4  32.106455   7.720896  0.960040  0.960040      0.939299  0.875916  0.379869   
 
    profitability  sustainability  dc_score  
 0       0.924894   

In [3]:
# Columns overview
print('Scores columns:', scores_df.columns.tolist())
print('Hex columns:', hex_df.columns.tolist())
print('\nCounts: scores', len(scores_df), 'hexes', len(hex_df))

Scores columns: ['region', 'lat', 'lon', 'raw_price', 'raw_load', 'raw_volatility', 'raw_peak', 'raw_renew', 'raw_temp', 'n_price', 'n_load', 'n_volatility', 'n_temp', 'n_renew', 'profitability', 'sustainability', 'dc_score']
Hex columns: ['hex_id', 'local_temp_c', 'elevation_m']

Counts: scores 13 hexes 1723


In [4]:
# Clean column names (already snake_case but enforce consistency)
score_cols = [c.strip() for c in scores_df.columns]
hex_cols = [c.strip() for c in hex_df.columns]

scores_df.columns = score_cols
hex_df.columns = hex_cols

# Ensure key columns exist
required_scores = {'region', 'lat', 'lon', 'profitability', 'sustainability', 'dc_score'}
required_hex = {'hex_id', 'local_temp_c', 'elevation_m'}
missing_scores = required_scores - set(scores_df.columns)
missing_hex = required_hex - set(hex_df.columns)
assert not missing_scores, f"Missing in scores: {missing_scores}"
assert not missing_hex, f"Missing in hex: {missing_hex}"

In [5]:
# Strategy for attaching region metrics to hexes:
# - We don't have an explicit region field per hex in the climate file.
# - The original folium map used nearest region centroids / dist_to_region.
# - We'll assign each hex to the nearest region centroid using haversine distance.

import numpy as np

def haversine_vec(lat1, lon1, lat2, lon2):
    # lat/lon in degrees, returns meters
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Prepare region centroids
region_centroids = scores_df[['region', 'lat', 'lon']].copy().rename(columns={'lat': 'region_lat', 'lon': 'region_lon'})

hex_coords = hex_df[['hex_id']].copy()

# If hex lat/lon are available in the map JSON, we could import them; for now, approximate using region lat/lon from the CSVs if present in hex_df
# (the provided hex_weather_data_all.csv only has temp/elevation). We'll require lat/lon per hex later when exporting GeoJSON.

# Placeholder: if hex_df already has lat/lon columns, keep them
for candidate in ['lat', 'lon']:
    if candidate in hex_df.columns:
        pass



In [6]:
# For a minimal merge now, map each hex to nearest region centroid.
# In the absence of hex lat/lon in hex_weather_data_all.csv, we cannot compute distances here.
# However, the existing score_map.json already includes lat/lon per hex. We'll read it to attach hex coordinates for distance mapping.

SCORE_MAP_JSON = Path('public/data/score_map.json')
assert SCORE_MAP_JSON.exists(), 'public/data/score_map.json not found (needed for hex lat/lon)'

with SCORE_MAP_JSON.open() as f:
    fc = json.load(f)

map_features = fc.get('features', [])
hex_lat_lon = []
for feat in map_features:
    props = feat.get('properties', {})
    hex_id = props.get('hex_id')
    lat = props.get('lat')
    lon = props.get('lon')
    if hex_id is not None and lat is not None and lon is not None:
        hex_lat_lon.append({'hex_id': hex_id, 'lat': lat, 'lon': lon})

hex_latlon_df = pd.DataFrame(hex_lat_lon).drop_duplicates(subset='hex_id')
print('Found hex lat/lon rows from map JSON:', len(hex_latlon_df))

# Merge lat/lon into hex climate df
hex_df = hex_df.merge(hex_latlon_df, on='hex_id', how='left')
missing_coords = hex_df['lat'].isna().sum()
print('Hex rows missing coords after merge:', missing_coords)

# Assign nearest region
regions_np = region_centroids[['region', 'region_lat', 'region_lon']].to_numpy()
hex_region = []
for _, row in hex_df.iterrows():
    lat = row['lat']
    lon = row['lon']
    if pd.isna(lat) or pd.isna(lon):
        hex_region.append({'region': None, 'dist_to_region_m': None})
        continue
    dists = haversine_vec(lat, lon, regions_np[:,1].astype(float), regions_np[:,2].astype(float))
    idx = np.argmin(dists)
    hex_region.append({'region': regions_np[idx,0], 'dist_to_region_m': float(dists[idx])})

hex_region_df = pd.DataFrame(hex_region)
hex_df = pd.concat([hex_df.reset_index(drop=True), hex_region_df], axis=1)

hex_df.head()

Found hex lat/lon rows from map JSON: 1723
Hex rows missing coords after merge: 0


,hex_id,local_temp_c,elevation_m,lat,lon,region,dist_to_region_m
0,0,27.067568,104.0,28.0,-82.0,FLA,0.0
1,1,26.656757,12.0,28.0,-82.0,FLA,0.0
2,2,27.183784,77.0,28.0,-82.0,FLA,0.0
3,3,23.048649,477.0,28.0,-82.0,FLA,0.0
4,4,25.024324,118.0,28.0,-82.0,FLA,0.0


In [7]:
# Join region metrics onto each hex using the assigned region
merged = hex_df.merge(scores_df, on='region', how='left', suffixes=('', '_region'))

# Use the merged frame as the working hex_df for downstream scoring
hex_df = merged.copy()

print('Merged rows:', len(hex_df))
hex_df.head()

Merged rows: 1723


,hex_id,local_temp_c,elevation_m,lat,lon,region,dist_to_region_m,lat_region,lon_region,raw_price,...,raw_renew,raw_temp,n_price,n_load,n_volatility,n_temp,n_renew,profitability,sustainability,dc_score
0,0,27.067568,104.0,28.0,-82.0,FLA,0.0,28.0,-82.0,84.0725,...,14.905253,21.280597,0.744803,0.744803,0.0,0.0,0.0,0.521362,0.0,0.208545
1,1,26.656757,12.0,28.0,-82.0,FLA,0.0,28.0,-82.0,84.0725,...,14.905253,21.280597,0.744803,0.744803,0.0,0.0,0.0,0.521362,0.0,0.208545
2,2,27.183784,77.0,28.0,-82.0,FLA,0.0,28.0,-82.0,84.0725,...,14.905253,21.280597,0.744803,0.744803,0.0,0.0,0.0,0.521362,0.0,0.208545
3,3,23.048649,477.0,28.0,-82.0,FLA,0.0,28.0,-82.0,84.0725,...,14.905253,21.280597,0.744803,0.744803,0.0,0.0,0.0,0.521362,0.0,0.208545
4,4,25.024324,118.0,28.0,-82.0,FLA,0.0,28.0,-82.0,84.0725,...,14.905253,21.280597,0.744803,0.744803,0.0,0.0,0.0,0.521362,0.0,0.208545


In [8]:
# Basic sanity checks
print('Nulls per important column:')
print(merged[['hex_id','region','profitability','sustainability','dc_score','local_temp_c','elevation_m','lat','lon','dist_to_region_m']].isna().sum())

# Quick descriptive stats for per-hex fields we care about
summary_cols = ['dc_score','profitability','sustainability','local_temp_c','elevation_m','dist_to_region_m']
print('\nDescribe:')
print(merged[summary_cols].describe())


Nulls per important column:
hex_id              0
region              0
profitability       0
sustainability      0
dc_score            0
local_temp_c        0
elevation_m         0
lat                 0
lon                 0
dist_to_region_m    0
dtype: int64

Describe:
          dc_score  profitability  sustainability  local_temp_c  elevation_m  \
count  1723.000000    1723.000000     1723.000000   1723.000000  1723.000000   
mean      0.584993       0.637101        0.550254      5.415026   658.091120   
std       0.200023       0.225168        0.273227      8.618531   688.405905   
min       0.208545       0.195956        0.000000    -19.437838   -57.000000   
25%       0.456543       0.585359        0.367141      1.222973   145.500000   
50%       0.656931       0.585359        0.450939      5.637838   373.000000   
75%       0.758318       0.826845        0.873624     11.290541  1037.000000   
max       0.887991       0.965919        0.873624     27.362162  3890.000000   

       

## Next steps
- Design per-hex adjustments (temperature/elevation tweaks) to create a unique `dc_score` per hex.
- Compute `dc_score_temp`/`temp_cool_score` analogs.
- Export FeatureCollection/JSON when ready (not done here).

## Per-hex scoring function
Approach:
- Base scores: region profitability/sustainability (already 60/40 combined)
- Temp bonus: cooler hexes get higher `temp_cool_score` (1 - normalized temp)
- Elevation bonus: higher elevation -> slight cooling proxy (normalized)
- Adjust sustainability with temp/elevation micro-weights; leave profitability mostly region-driven (can add small temp effect if desired)
- Optional spatial smoothing: k-NN average to reduce abrupt jumps (kept small)

In [9]:
def minmax_clamped(series, lower_q=0.01, upper_q=0.99):
    lo, hi = series.quantile(lower_q), series.quantile(upper_q)
    rng = hi - lo if hi > lo else 1
    return ((series.clip(lo, hi) - lo) / rng).fillna(0.5)

# Normalized climate signals (0-1)
hex_df['temp_norm'] = minmax_clamped(hex_df['local_temp_c'])
hex_df['temp_cool_score'] = 1 - hex_df['temp_norm']  # cooler is better
hex_df['elev_norm'] = minmax_clamped(hex_df['elevation_m'])

# Micro-weights tuned for smoother transitions
TEMP_WEIGHT = 0.15  # contributes to sustainability
ELEV_WEIGHT = 0.05  # smaller effect
SMOOTH_K = 20       # neighbors for smoothing
SMOOTH_BLEND = 0.25 # self vs neighbor blend

# Adjust sustainability using climate signals
hex_df['sustainability_hex'] = (
    hex_df['sustainability']
    + hex_df['temp_cool_score'] * TEMP_WEIGHT
    + hex_df['elev_norm'] * ELEV_WEIGHT
)

# Profitability mostly region-level; leave as-is (optional: add tiny temp effect)
hex_df['profitability_hex'] = hex_df['profitability']

# Recombine to per-hex dc_score (60/40 weighting)
hex_df['dc_score_hex'] = 0.6 * hex_df['sustainability_hex'] + 0.4 * hex_df['profitability_hex']

hex_df[['hex_id','region','profitability','profitability_hex','sustainability','sustainability_hex','dc_score','dc_score_hex']].head()


,hex_id,region,profitability,profitability_hex,sustainability,sustainability_hex,dc_score,dc_score_hex
0,0,FLA,0.521362,0.521362,0.0,0.001560,0.208545,0.209481
1,1,FLA,0.521362,0.521362,0.0,0.000180,0.208545,0.208653
2,2,FLA,0.521362,0.521362,0.0,0.001155,0.208545,0.209238
3,3,FLA,0.521362,0.521362,0.0,0.007156,0.208545,0.212838
4,4,FLA,0.521362,0.521362,0.0,0.001770,0.208545,0.209607


In [10]:
# Optional spatial smoothing for dc_score_hex
import numpy as np

coords = hex_df[['lat','lon']].to_numpy()
scores_arr = hex_df['dc_score_hex'].to_numpy()

def knn_smooth(coords, values, k=6, self_weight=0.7):
    smoothed = np.array(values, copy=True)
    valid = ~np.isnan(coords).any(axis=1)
    if valid.sum() < 2:
        return smoothed
    k_eff = max(1, min(k, valid.sum() - 1))
    coords_v = coords[valid]
    vals_v = values[valid]

    # haversine distances for smoother spatial blending
    lat = np.radians(coords_v[:,0])[:,None]
    lon = np.radians(coords_v[:,1])[:,None]
    lat_T = lat.T
    lon_T = lon.T
    dphi = lat - lat_T
    dlambda = lon - lon_T
    a = np.sin(dphi/2.0)**2 + np.cos(lat)*np.cos(lat_T)*np.sin(dlambda/2.0)**2
    dist = 2 * 6371000 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    np.fill_diagonal(dist, np.inf)

    knn_idx = np.argpartition(dist, kth=k_eff-1, axis=1)[:, :k_eff]
    neighbor_means = vals_v[knn_idx].mean(axis=1)
    smoothed_v = self_weight * vals_v + (1 - self_weight) * neighbor_means
    smoothed[valid] = smoothed_v
    return smoothed

hex_df['dc_score_hex_smooth'] = knn_smooth(coords, scores_arr, k=SMOOTH_K, self_weight=SMOOTH_BLEND)
hex_df[['dc_score_hex','dc_score_hex_smooth']].describe()


,dc_score_hex,dc_score_hex_smooth
count,1723.000000,1723.000000
mean,0.612440,0.611642
std,0.206227,0.205394
min,0.208563,0.209174
25%,0.464125,0.467766
50%,0.690525,0.690329
75%,0.795057,0.792709
max,0.921598,0.920189


In [11]:
# Summaries to verify per-hex variation
summary_cols = [
    'dc_score','dc_score_hex','dc_score_hex_smooth',
    'sustainability','sustainability_hex',
    'profitability','profitability_hex',
    'temp_cool_score','elev_norm',
]
print(hex_df[summary_cols].describe())

# Inspect a few neighboring hexes (random sample)
hex_df.sample(5)[['hex_id','region','lat','lon','dc_score_hex','dc_score_hex_smooth','temp_cool_score','elev_norm']]


          dc_score  dc_score_hex  dc_score_hex_smooth  sustainability  \
count  1723.000000   1723.000000          1723.000000     1723.000000   
mean      0.584993      0.612440             0.611642        0.550254   
std       0.200023      0.206227             0.205394        0.273227   
min       0.208545      0.208563             0.209174        0.000000   
25%       0.456543      0.464125             0.467766        0.367141   
50%       0.656931      0.690525             0.690329        0.450939   
75%       0.758318      0.795057             0.792709        0.873624   
max       0.887991      0.921598             0.920189        0.873624   

       sustainability_hex  profitability  profitability_hex  temp_cool_score  \
count         1723.000000    1723.000000        1723.000000      1723.000000   
mean             0.595999       0.637101           0.637101         0.449314   
std              0.285395       0.225168           0.225168         0.221224   
min              0.000

,hex_id,region,lat,lon,dc_score_hex,dc_score_hex_smooth,temp_cool_score,elev_norm
182,182,TEX,31.0,-99.0,0.237108,0.238718,0.171762,0.047633
1391,1391,NW,45.5,-120.5,0.781689,0.784171,0.486898,0.000000
1401,1401,NW,45.5,-120.5,0.795445,0.793800,0.634518,0.277919
1716,1716,NW,45.5,-120.5,0.803475,0.799421,0.935152,0.011252
409,409,CAL,36.5,-119.5,0.470634,0.472578,0.172812,0.241539


Next: export a FeatureCollection/JSON using these per-hex scores, or feed into the D3 map. (Not done here.)